# Library

## Python packages 

In [1]:
import pandas as pd
import numpy as np
import regex as re
import rpy2
import os
import functools as fct
from collections import Counter
import pickle 

/Users/tanya/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## R packages 

In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
library(rlang)
library(ggplot2)
library(dplyr)
library(scales)
library(reshape2)
library(cowplot)
library(treemapify)

Use suppressPackageStartupMessages() to eliminate package startup
messages

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



## Set paths 

In [14]:
path_to_vcfs = '/Users/tanya/Documents/OTT/Conflicts_ClinVar/data'   # path to initial ClinVar dfs
path_to_gatk = '/Users/tanya/Documents/tools/gatk-4.5.0.0/gatk'
path_to_tables = '/Users/tanya/Documents/OTT/Conflicts_ClinVar/tables'
path_to_plots = '/Users/tanya/Documents/OTT/Conflicts_ClinVar/plots'
path_to_temp = '/Users/tanya/Documents/OTT/Conflicts_ClinVar/temp'


path_to_data = '/Users/tanya/Documents/OTT/Conflicts_ClinVar/data' 

## Load data 

In [5]:
with open(f'{path_to_temp}/coi_clnsig.pkl', 'rb') as file:  # load coi_clnsig dict
    coi_clnsig = pickle.load(file)
    
with open(f'{path_to_temp}/cv_dict.pkl', 'rb') as file:  # load main dict
    cv_dict = pickle.load(file)

# Collect data to VEP annotation

### COI

In [6]:
# List of IDs to filter by

coi_ids_list = coi_clnsig.keys() 

# Filter rows based on ID from the list of COI

coi_coords = cv_dict['clinvar_20240407'][cv_dict['clinvar_20240407']['ID'].isin(coi_ids_list)]

coi_coords = coi_coords[['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER']]
coi_coords.columns = ['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER']
coi_coords['INFO'] = ''

print(coi_coords.shape)

(131280, 8)


In [ ]:
coi_coords.to_csv(f'{path_to_temp}/COI.vcf', sep='\t', index=False)

! cat ./temp/header.vcf ./temp/COI.vcf > ./temp/coi_to_VEP.vcf
! rm ./temp/COI.vcf

In [ ]:
# cmd-line VEP annotation
# PATH = /media/DATA/tl_projects/conflicts_clinvar/vcfs

docker run --user {id_u:id_g} -v '/media/DATA/ott_ngs/data/grch38/vep':'/vep_data' \
        -v '/media/DATA/tl_projects/conflicts_clinvar/vcfs':'/vcfs' \
        -v '/media/DATA/ott_ngs/data/grch38/broad_ref':'/fasta' ensemblorg/ensembl-vep vep \
        --cache --offline --format vcf --vcf --force_overwrite --force --refseq  --af \
        --af_gnomadg --max_af --hgvs  --no_escape --canonical \
        --fasta /fasta/Homo_sapiens_assembly38.fasta \
        --dir_cache /vep_data \
        --dir_plugins /vep_data/Plugins \
        --custom /vep_data/clinvar_20230930.vcf.gz,ClinVar,vcf,exact,0,CLNSIG,CLNSIGCONF,CLNREVSTAT,CLNDN,CLNDISDBINCL  \
        --custom /vep_data/ruseq.sites.v1.1.vcf.gz,LOCALAFDB,vcf,exact,0,AF_healthy \
        --custom /vep_data/CosmicCodingMuts.vcf.gz,COSMIC,vcf,exact,0 \
        --plugin dbNSFP,/vep_data/dbNSFP4.1a.txt.gz,PROVEAN_pred,SIFT_pred,Polyphen2_HVAR_pred \
        --input_file /vcfs/coi_to_VEP.vcf  \
        --output_file /vcfs/COI.VEP.vcf

In [ ]:
! bgzip -c COI.VEP.vcf > COI.VEP.vcf.gz
! tabix -p vcf COI.VEP.vcf.gz

In [ ]:
# bcftools +split-vep

! bcftools +split-vep -s worst -f '%CHROM %POS %ID %REF %ALT %SYMBOL %IMPACT %Consequence %MAX_AF %MAX_AF_POPS %gnomADg_AF %gnomADg_AFR_AF %gnomADg_AMI_AF %gnomADg_AMR_AF %gnomADg_ASJ_AF %gnomADg_EAS_AF %gnomADg_FIN_AF %gnomADg_MID_AF %gnomADg_NFE_AF %gnomADg_OTH_AF %gnomADg_SAS_AF %LOCALAFDB_AF_healthy\n' COI.VEP.vcf.gz

In [10]:
! awk -v OFS="\t" '$1=$1' ./temp/COI.VEP.csv > ./temp/COI1.VEP.csv
! rm ./temp/COI.VEP.csv
! mv ./temp/COI1.VEP.csv ./temp/COI.VEP.csv

### noCOI 

In [ ]:
no_coi_coords =  cv_dict['clinvar_20240407'][~cv_dict['clinvar_20240407']['ID'].isin(coi_ids_list)]

no_coi_coords = no_coi_coords[['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL','FILTER']]
no_coi_coords.columns = ['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER']
no_coi_coords['INFO'] = ''
print(no_coi_coords.shape)

In [ ]:
# cmd-line VEP annotation
# PATH = /media/DATA/tl_projects/conflicts_clinvar/vcfs

docker run --user 1003:1001 -v '/media/DATA/ott_ngs/data/grch38/vep':'/vep_data' \
        -v '/media/DATA/tl_projects/conflicts_clinvar/vcfs':'/vcfs' \
        -v '/media/DATA/ott_ngs/data/grch38/broad_ref':'/fasta' ensemblorg/ensembl-vep vep \
        --cache --offline --format vcf --vcf --force_overwrite --force --refseq  --af \
        --af_gnomadg --max_af --hgvs  --no_escape --canonical \
        --fasta /fasta/Homo_sapiens_assembly38.fasta \
        --dir_cache /vep_data \
        --dir_plugins /vep_data/Plugins \
        --custom /vep_data/clinvar_20230930.vcf.gz,ClinVar,vcf,exact,0,CLNSIG,CLNSIGCONF,CLNREVSTAT,CLNDN,CLNDISDBINCL  \
        --custom /vep_data/ruseq.sites.v1.1.vcf.gz,LOCALAFDB,vcf,exact,0,AF_healthy \
        --custom /vep_data/CosmicCodingMuts.vcf.gz,COSMIC,vcf,exact,0 \
        --plugin dbNSFP,/vep_data/dbNSFP4.1a.txt.gz,PROVEAN_pred,SIFT_pred,Polyphen2_HVAR_pred \
        --input_file /vcfs/nocoi_to_VEP.vcf  \
        --output_file /vcfs/no_coi_20230930.VEP.vcf

In [ ]:
# 

! bgzip -c  bgzip -c nocoi_20240407.VEP.vcf > nocoi_20240407.VEP.vcf.gz
! tabix -p vcf nocoi_20240407.VEP.vcf.gz

In [ ]:
# bcftools +split-vep

! bcftools +split-vep -s worst -f '%CHROM %POS %ID %REF %ALT %SYMBOL %IMPACT %Consequence %MAX_AF %MAX_AF_POPS %gnomADg_AF %gnomADg_AFR_AF %gnomADg_AMI_AF %gnomADg_AMR_AF %gnomADg_ASJ_AF %gnomADg_EAS_AF %gnomADg_FIN_AF %gnomADg_MID_AF %gnomADg_NFE_AF %gnomADg_OTH_AF %gnomADg_SAS_AF %LOCALAFDB_AF_healthy\n' nocoi_20240407.VEP.vcf.gz > nocoi_20240407.VEP.csv

In [ ]:
! awk -v OFS="\t" '$1=$1' no_coi_20230930.VEP.csv > noCOI_20230930.VEP.csv

## merge after VEP annotation

In [7]:
# Load COI df

coi_vep = pd.read_csv(f'{path_to_tables}/COI.VEP.csv', sep='\t', header=None)
coi_vep.columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'SYMBOL', 'IMPACT', 'Consequence', 'MAX_AF', 'MAX_AF_POPS',
                   'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 
                   'gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF',
                   'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'LOCALAFDB_AF_healthy']

# add molecular change information

add_info = cv_dict['clinvar_20240407'][['ID','CLNVC', 'CLNDISDB']]
coi_vep = coi_vep.merge(add_info, on=['ID'], how='left')

# mark COI
coi_vep['CLNSIG_MASK'] = 'COI'
print(coi_vep.shape)
coi_vep.head(3)

/var/folders/sy/0s6mz48x25xdw7n08d30rp5w0000gn/T/ipykernel_34557/1420700278.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  coi_vep = pd.read_csv(f'{path_to_tables}/COI.VEP.csv', sep='\t', header=None)


(134238, 25)


,CHROM,POS,ID,REF,ALT,SYMBOL,IMPACT,Consequence,MAX_AF,MAX_AF_POPS,...,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,LOCALAFDB_AF_healthy,CLNVC,CLNDISDB,CLNSIG_MASK
0,1,930200,1043045,G,A,SAMD11,MODERATE,missense_variant,0.001348,gnomADg_EAS,...,0.001348,0,0,1.47e-05,0,0,.,NaN,NaN,COI
1,1,935839,1085785,C,T,SAMD11,MODERATE,missense_variant,0.005476,gnomADg_ASJ,...,0,0,0,0.0001029,0,0,.,NaN,NaN,COI
2,1,939117,1427749,G,A,SAMD11,MODERATE,missense_variant,0.0004776,gnomADg_OTH,...,0,0,0,2.94e-05,0.0004776,0,.,NaN,NaN,COI


In [8]:
# Load non-COI df

nocoi_vep = pd.read_csv(f'{path_to_tables}/nocoi_20240407.VEP.csv', sep='\t', header=None)
nocoi_vep.columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'SYMBOL', 'IMPACT', 'Consequence', 
                       'MAX_AF', 'MAX_AF_POPS', 'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 
                       'gnomADg_AMR_AF','gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF',
                   'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'LOCALAFDB_AF_healthy']

# add molecular change information and clnsig

add_info = cv_dict['clinvar_20240407'][['ID','CLNVC', 'CLNDISDB','CLNSIG_MASK']]
nocoi_vep = nocoi_vep.merge(add_info, on=['ID'], how='left')
print(nocoi_vep.shape)
nocoi_vep.head(3)

/var/folders/sy/0s6mz48x25xdw7n08d30rp5w0000gn/T/ipykernel_34557/3745659847.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  nocoi_vep = pd.read_csv(f'{path_to_tables}/nocoi_20240407.VEP.csv', sep='\t', header=None)


(2591000, 25)


,CHROM,POS,ID,REF,ALT,SYMBOL,IMPACT,Consequence,MAX_AF,MAX_AF_POPS,...,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,LOCALAFDB_AF_healthy,CLNVC,CLNDISDB,CLNSIG_MASK
0,1,69134,2205837,A,G,OR4F5,MODERATE,missense_variant,0.08805,gnomADe_ASJ,...,0,0,0,0.0002006,0,0.003012,.,NaN,NaN,NaN
1,1,69581,2252161,C,G,OR4F5,MODERATE,missense_variant,.,.,...,.,.,.,.,.,.,.,NaN,NaN,NaN
2,1,69682,2396347,G,A,OR4F5,MODERATE,missense_variant,0.000631,gnomADe_AMR,...,0,0,0,0,0,0,.,NaN,NaN,NaN


In [9]:
# concatenate dfs

vep = pd.concat([coi_vep,nocoi_vep])
vep.shape

(2725238, 25)

In [15]:
# filter by MIM genes

# BioMart data
gene_mim = pd.read_csv(f'{path_to_data}/features/bioMart_Gene_MIM.txt', sep='\t', keep_default_na=False)

gene_mim = gene_mim[gene_mim['MIM morbid accession']!= '']
gene_mim_glist = list(gene_mim['Gene name'].drop_duplicates())

vep = vep[vep['SYMBOL'].isin(gene_mim_glist)]
vep.shape

(2296245, 25)

In [20]:
vep.to_csv(f'{path_to_tables}/VEP.csv', sep=',', index = False)